# Tipado fuerte

In [31]:
x = 2
y = pi
x + y -- el compilador se da cuenta solo

5.141592653589793

In [36]:
x :: Int
x = 2
y = pi
x + y

<interactive>:1:5: error:
    • No instance for (Floating Int) arising from a use of ‘y’
    • In the second argument of ‘(+)’, namely ‘y’
      In the expression: x + y
      In an equation for ‘it’: it = x + y

In [38]:
fromIntegral x + y

5.141592653589793

# Lazyness

Por default, todo es __lazy__ en Haskell

In [1]:
factorial 0 = 1
factorial n = n * factorial (n-1)

x :: [Int]
x = map factorial [1..10]

In [ ]:
:sprint x
-- x = _

In [2]:
length x

10

In [ ]:
:sprint x
-- x = [_,_,_,_,_,_,_,_,_,_]

Para calcular el length, no hace falta saber qué hay en la lista, sólo cuántas cosas hay

In [4]:
x!!9

3628800

In [ ]:
:sprint x
-- x = [_,_,_,_,_,_,_,_,_,3628800]

Sólo se calculó el factorial de 10

# Recursividad

Importante en Haskell por ser un lenguaje funcional: no se define cómo __hacer__ las cosas sino cómo __son__ las cosas.

Ejemplo: máximo de una lista

In [11]:
maxim :: (Ord a) => [a] -> a
maxim [] = error "Máximo de una lista vacía"
maxim [x] = x
maxim (x:xs)
    | x > maxTail = x
    | otherwise   = maxTail
    where maxTail = maxim xs

print (maxim ['a','b','z','f','h'])

'z'

Ejemplo: `repl` recibe un numero `n` y valor de cualquier tipo. Devuelve una lista del valor replicado `n` veces

In [13]:
repl :: Int -> a -> [a]
repl n x
    | n <= 0    = []
    | otherwise = x:repl (n-1) x

print(repl 5 "Haskell")

["Haskell","Haskell","Haskell","Haskell","Haskell"]

## Comparando con Java

### Quicksort Java

In [ ]:
package com.java2novice.sorting;
 
public class MyQuickSort {
     
    private int array[];
    private int length;
 
    public void sort(int[] inputArr) {
         
        if (inputArr == null || inputArr.length == 0) {
            return;
        }
        this.array = inputArr;
        length = inputArr.length;
        quickSort(0, length - 1);
    }
 
    private void quickSort(int lowerIndex, int higherIndex) {
         
        int i = lowerIndex;
        int j = higherIndex;
        // calculate pivot number, I am taking pivot as middle index number
        int pivot = array[lowerIndex+(higherIndex-lowerIndex)/2];
        // Divide into two arrays
        while (i <= j) {
            /**
             * In each iteration, we will identify a number from left side which 
             * is greater then the pivot value, and also we will identify a number 
             * from right side which is less then the pivot value. Once the search 
             * is done, then we exchange both numbers.
             */
            while (array[i] < pivot) {
                i++;
            }
            while (array[j] > pivot) {
                j--;
            }
            if (i <= j) {
                exchangeNumbers(i, j);
                //move index to next position on both sides
                i++;
                j--;
            }
        }
        // call quickSort() method recursively
        if (lowerIndex < j)
            quickSort(lowerIndex, j);
        if (i < higherIndex)
            quickSort(i, higherIndex);
    }
 
    private void exchangeNumbers(int i, int j) {
        int temp = array[i];
        array[i] = array[j];
        array[j] = temp;
    }
     
    public static void main(String a[]){
         
        MyQuickSort sorter = new MyQuickSort();
        int[] input = {24,2,45,20,56,75,2,56,99,53,12};
        sorter.sort(input);
        for(int i:input){
            System.out.print(i);
            System.out.print(" ");
        }
    }
}

### Quicksort Haskell 

In [20]:
quicksort :: (Ord a) => [a] -> [a]
quicksort [] = []
quicksort (x:xs) =
    let smallerSorted = quicksort [a | a <- xs, a <= x]
        biggerSorted  = quicksort [a | a <- xs, a > x]
    in  smallerSorted ++ [x] ++ biggerSorted

quicksort [24,2,45,20,56,75,2,56,99,53,12]

[2,2,12,20,24,45,53,56,56,75,99]

# Funciones currificadas


Cada función de Haskell toma __sólo un parametro__. 

¿Cómo hacemos para llamar a una función con mas de un parametro? Currificamos la función

Ejemplo: multiplicación de 3 argumentos

In [ ]:
multThree :: (Num a) => a -> a -> a -> a
multThree x y z = x * y * z

Se puede leer como que `multThree` toma un `x` y devuelve otra función que recibe `y`. Esa función a su vez devuelve otra función que recibe `z` y devuelve el resultado (la multiplicación de los 3). Se puede reescribir como:

In [ ]:
multThree :: (Num a) => a -> (a -> (a -> a))

Pasando en limpio ... se van componiendo funciones que reciben un numero y devuelven otra función que a su vez recibe un numero y devuelve otro:

In [15]:
:t multThree 

multThree :: forall a. Num a => a -> a -> a -> a

In [16]:
:t multThree 1

multThree 1 :: forall a. Num a => a -> a -> a

In [17]:
:t multThree 1 2

multThree 1 2 :: forall a. Num a => a -> a

¿Beneficios? Si llamamos a una función con parametros de menos, obtenemos una función __parcialmente aplicada__, que puede ser utilizada para crear otra función. 

Ejemplo: una función que recibe dos numeros y los multiplica por 9, utilizando la función `multThree`

In [14]:
multThree :: (Num a) => a -> a -> a -> a
multThree x y z = x * y * z

multTwoWithNine = multThree 9
multTwoWithNine 2 3

54

# Funciones de Orden Superior

Como vimos recién, las funciones pueden tomar funciones como parámetros y devolver otras funciones.

Ejemplo: `applyTwice` recibe una función y un número, y devuelve la doble aplicación del número a la función.

In [18]:
applyTwice :: (a -> a) -> a -> a
applyTwice f x = f (f x)

Utilizando la función `multThree` que habíamos definido antes:

In [19]:
applyTwice :: (a -> a) -> a -> a
applyTwice f x = f (f x)

multThree :: (Num a) => a -> a -> a -> a
multThree x y z = x * y * z

applyTwice (multThree 2 2) 3

48